In [3]:
import torch
from torch_geometric.datasets import Planetoid

In [17]:
dataset = Planetoid(root='/tmp/Cora', name='Cora')

In [18]:
class GNN(torch.nn.Module):
    def __init__(self, input_embed_dim : int, output_embed_dim : int, activation=torch.nn.Sigmoid):
        super(GNN, self).__init__()
        # The k-1 embedding dimensions.
        self.input_embed_dim = input_embed_dim
        # The k embedding dimensions.
        self.output_embed_dim = output_embed_dim
        self.activation = activation
        # The neighbourhood weights.
        self.W_neigh = torch.nn.Parameter(torch.rand(self.input_embed_dim, self.output_embed_dim))
        # The self weights.
        self.W_self = torch.nn.Parameter(torch.rand(self.input_embed_dim, self.output_embed_dim))
        self.sigmoid = torch.nn.Sigmoid()
        self.reset_parameters()

    def reset_parameters(self):
        torch.nn.init.xavier_uniform_(self.W_neigh)
        torch.nn.init.xavier_uniform_(self.W_self)

    def forward(self, H : torch.Tensor, A : torch.Tensor):
        if A.shape[0] != A.shape[1]:
          raise Exception("GNN layers expects an Adjecancy Matrix(square).")
        if H.shape[0] != A.shape[0]:
          raise Exception("Shape mismatch between H and A. expected shape (", A.shape[0], self.input_embed_dim, ") got (", H.shape[0], ", ", H.shape[1], ")")
        if H.shape[1] != self.input_embed_dim:
          raise Exception("H expected shape (", A.shape[0], ", ", self.input_embed_dim, ") got (", H.shape[0], ", ", H.shape[1], ")")

        return self.activation(torch.matmul(torch.matmul(A, H), self.W_neigh) + torch.matmul(H, self.W_self))

In [ ]:
class GNNSelfLoop(torch.nn.Module):
    def __init__(self, input_embed_dim : int, output_embed_dim : int):
        super(GNN, self).__init__()
        # The k-1 embedding dimensions.
        self.input_embed_dim = input_embed_dim
        # The k embedding dimensions.
        self.output_embed_dim = output_embed_dim

        # The shared weights.
        self.W = torch.nn.Parameter(torch.rand(self.input_embed_dim, self.output_embed_dim))
        self.sigmoid = torch.nn.Sigmoid()
        self.reset_parameters()

    def reset_parameters(self):
        torch.nn.init.xavier_uniform_(self.W)

    def forward(self, H : torch.Tensor, A : torch.Tensor):
        if A.shape[0] != A.shape[1]:
          raise Exception("GNN layers expects an Adjecancy Matrix(square).")
        if H.shape[0] != A.shape[0]:
          raise Exception("Shape mismatch between H and A. expected shape (", A.shape[0], self.input_embed_dim, ") got (", H.shape[0], ", ", H.shape[1], ")")
        if H.shape[1] != self.input_embed_dim:
          raise Exception("H expected shape (", A.shape[0], ", ", self.input_embed_dim, ") got (", H.shape[0], ", ", H.shape[1], ")")
        AI = A + torch.eye(A.shape[0])
        return self.sigmoid(torch.matmul(torch.matmul(AI, H), self.W_neigh))

In [19]:
device = torch.device('cuda')

In [20]:
A = torch.zeros(dataset.x.shape[0], dataset.x.shape[0], requires_grad=False)
edge_index = dataset.edge_index
for i in range (0, edge_index.shape[1]):
  A[edge_index[0, i], edge_index[1, i]] += 1
A = A.to(device)
H = dataset.x.to(device)

In [22]:
class SemiSupervisedClassifier(torch.nn.Module):
    def __init__(self, input_embed_dim : int,  num_classes : int, latent_dim = None):
        super(SemiSupervisedClassifier, self).__init__()
        if latent_dim is None:
          latent_dim = input_embed_dim
        self.gnn1 = GNN(input_embed_dim, latent_dim, torch.nn.ReLU())
        self.gnn2 = GNN(latent_dim, num_classes, torch.nn.Softmax(dim=1))

    def forward(self, H : torch.Tensor, A : torch.Tensor):
        return self.gnn2(self.gnn1(H, A), A)

In [23]:
model = SemiSupervisedClassifier(dataset.x.shape[1], dataset.num_classes).to(torch.device('cuda'))

In [14]:
import torch.nn.functional as F

In [24]:
optimizer = torch.optim.Adam(model.parameters(), lr=0.01, weight_decay=5e-4)

model.train()
for epoch in range(60):
    optimizer.zero_grad()
    out = model(H, A)
    loss = F.nll_loss(out[dataset.train_mask], dataset.y[dataset.train_mask].to(device))
    print(loss)
    loss.backward()
    optimizer.step()

tensor(-0.1387, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(-0.6146, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(-0.6617, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(-0.7300, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(-0.7684, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(-0.8459, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(-0.8612, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(-0.8855, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(-0.8460, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(-0.8411, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(-0.8670, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(-0.8690, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(-0.8768, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(-0.8919, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(-0.8857, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(-0.8902, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(-0.9068, device='

In [25]:
model.eval()
pred = model(H, A).argmax(dim=1)
correct = (pred[dataset.test_mask] == dataset.y[dataset.test_mask].to(device)).sum()
acc = int(correct) / int(dataset.test_mask.sum())
print(f'Accuracy: {acc:.4f}')

Accuracy: 0.6770
